<font size="6">Модели на основе энкодера Трансформера</font>

Архитектура классического Трансформера состоит из энкодера и декодера. Она применяется в задачах машинного перевода и других задачах преобразования последовательностей, где входная и выходная последовательности могут иметь разную длину (sequence-to-sequence).

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L06/out/transformer_architecture.png" width="450"></center>

<center><em>Архитектура трансформера</em></center>

<center><em>Источник: <a href="https://arxiv.org/abs/1706.03762">Attention Is All You Need</a></em></center>

Однако блоки энкодера и декодера можно использовать по отдельности.
- Модели на основе декодера применяются для генерации текста и используют маскированное внимание (Generative Pre-trained Transformers, GPT)
- Модели на основе энкодера применяются для других задач: классификации одного или пары предложений, теггирования последовательности, поиска ответа на вопрос (Bidirectional Encoder Representations from Transformers, BERT)

Сегодня мы подробно рассмотрим архитектуру модели BERT и её применение для различных задач. BERT возник как результат исправления недочетов предыдущих моделей, поэтому рассказ про него мы начнем немного издалека.

# Первая модель на улице Сезам — ELMo

Модель ELMo была представлена в статье:
* [[paper] 🎓 Peters M. E. et al. (2018). Deep contextualized word representations](https://arxiv.org/abs/1802.05365)

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/elmo.png" width="600"></center>

<center><em>Источник: <a href="https://arxiv.org/abs/1802.05365">Deep contextualized word representations</a></em></center>

Различным значениям слова *play* соответствуют разные контексты
употребления. Нужно передавать не только значение слова, но и контекстуальную
информацию — **контекстуализированные векторные
представления слов**. Контекстуализированные эмбеддинги присваивают словам разные векторы на основе их семантики в контексте предложения. Такие контекстуализированные векторы вычисляются посредством обучения языковой модели: ELMo = **E**mbeddings from **L**anguage **Mo**dels.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/elmo_2.png" width="800"></center>

<center><em>Источник: <a href="https://arxiv.org/abs/1810.04805">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding</a></em></center>

Для обучения векторов ELMo используется двунаправленная языковая модель (bidirectional Language Model или biLM), основанная на рекуррентных нейронных сетях.

Модель вычисляет вероятность появления последовательности токенов $t_1, t_2, \dots, t_N$ на основе двух направлений обработки текста:
- прямой (forward):
  - учитывается информация об определенном слове и контексте перед ним
  - оценивается вероятность $t_k$ при условии предшествующего контекста $t_1, ..., t_{k-1}$
- обратный (backward):
  - учитывается информация о слове и контексте после него
  - оценивается вероятность $t_k$ при условии последующего контекста $t_{k+1}, \dots, t_N$

Важно: ELMo не имеет отношения к Трансформерам.

Модель состоит из двух слоев, каждый из которых представляет собой двунаправленную сеть долгой краткосрочной памяти (biLSTM).

Информация из прямого и обратного прохода на первом слое формирует промежуточные векторные представления слов, которые подаются на вход второго слоя модели.

Результирующие векторы — взвешенная сумма необработанных эмбеддингов слов и двух промежуточных векторных представлений после первого и второго слоев biLSTM.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/elmo_architecture.png" width="600"></center>

<center><em>Источник: <a href="https://www.researchgate.net/publication/359157231_Identifying_Contradictions_in_the_Legal_Proceedings_Using_Natural_Language_Models">Identifying Contradictions in the Legal Proceedings Using Natural Language Models</a></em></center>

Поскольку модель обучается на задаче языкового моделирования, ей не требуются размеченные тексты, появляется возможность использовать большие объемы данных для обучения. Модель выучивает некоторые общие знания о языке, не привязываясь к какой-либо конкретной задаче.

ELMo стала важным шагом к распространению трансферного обучения в области NLP. Выходы модели ELMo могут использоваться как контекстуализированные эмбеддинги для различных задач обработки текста.

В случае word2vec каждому слову соответствует один конкретный фиксированный  вектор, и эти векторы слов можно сохранять и загружать при необходимости.
ELMo же строит контекстно зависимые векторы. Чтобы получить векторы слов предложения, нужно сначала пропустить всё это предложение через модель. При этом обучение уже не производится — модель работает в режиме извлечения признаков.

Это позволяет учитывать многозначность слов. Например, для слова *bank* ELMo создаст разные векторы в зависимости от контекста: один — отражающий значение "берег реки", другой — для значения "финансовая организация".

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/bank_embeddings.png" width="600"></center>

<center><em>Источник: <a href="https://github.com/hengluchang/visualizing_contextual_vectors">Visualizing ELMo Contextual Vectors (code for generating figure)</a></em></center>

# Вторая модель на улице Сезам — BERT

Модель BERT была представлена в статье:
* [[paper] 🎓 Devlin J. et al. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)

Идеи, которые были предложены ранее и удачно объединились при создании модели BERT:
- предобучение на задаче языкового моделирования, которая не требует разметки, и дообучение под конкретные задачи — ELMo
- механизм множественного внутреннего внимания, используемый без RNN, — Трансформер
- использование только одной части модели Трансформера (декодер) — GPT

Недостаток ELMo: анализируя левый и правый контекст отдельно с помощью biLSTM, мы можем терять часть информации. Хотелось бы учитывать левый и правый контекст одновременно.

Новшество BERT — использование **энкодера** Трансформера, чтобы получить "обогащенные" вниманием векторы слов.


<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/bert.png" width="600"></center>

<center><em>Источник: <a href="https://botpenguin.com/glossary/bert">What is BERT?</a></em></center>

BERT состоит из нескольких последовательно соединенных блоков энкодера трансформера.

На вход модель получает последовательность токенов, на выходе отдает векторное представление для каждого токена, обогащенное контекстом. Энкодер содержит механизм внутреннего внимания (Self-Attention), который применяется к каждому токену и позволяет улавливать контекст.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/bert_input.png" width="800"></center>

<center><em>Источник: <a href="https://jalammar.github.io/illustrated-bert/">The Illustrated BERT</a></em></center>

Две конфигурации:
- базовая (base): 12 слоев, размер скрытого слоя — 768, количество параметров — 110 миллионов
- расширенная (large): 24 слоя, размер скрытого слоя — 1024, количество параметров — 340 миллионов

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/bert_base_large.png" width="800"></center>

<center><em>Источник: <a href="https://jalammar.github.io/illustrated-bert/">The Illustrated BERT</a></em></center>

BERT обучается на двух задачах:
- маскированное языковое моделирование (masked language modeling, MLM)
- предсказание следующего предложения (next sentence prediction, NSP)

### Маскированное языковое моделирование

15% случайно выбранных токенов по всему корпусу маскируется — заменяется на спецтокен `[MASK]`. Задача модели — предсказать наиболее вероятный токен на месте маски.

Если модель всегда должна будет предсказывать наиболее вероятное слово только для масок, то для остальных слов векторы будут обучаться хуже. Нужно "обмануть" модель, чтобы она смотрела на все слова входной последовательности.

Среди выбранных 15% токенов:
- 80% маскируются: my dog is `[MASK]`
- 10% меняются на случайное слово: my dog is apple
- 10% остаются: my dog is hairy

Это разбиение меняется на каждой эпохе обучения.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/masked_language_modeling.png" width="800"></center>

<center><em>Источник: <a href="https://jalammar.github.io/illustrated-bert/">The Illustrated BERT</a></em></center>

Обучение на задаче маскированного языкового моделирования позволяет получить контекстуализированные векторы токенов. Предобученные векторы можно использовать в других задачах обработки текста.

### Предсказание следующего предложения

Помимо векторов слов, хотелось бы получать также векторные представления предложений. Для этого попробуем предсказать, следует ли одно предложение за другим.

Предложения разделены спецтокеном `[SEP]`. За классификацию отвечает спецтокен `[CLS]`. Он содержит представление обо всем предложении. Выход `[CLS]` токена пропускается через линейный слой размера 2.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/next_sentence_prediction.png" width="800"></center>

<center><em>Источник: <a href="https://jalammar.github.io/illustrated-bert/">The Illustrated BERT</a></em></center>

Положительные примеры представляют собой предложения, которые действительно следуют друг за другом в корпусе.

Вход: `[CLS] the man went to the [MASK] store [SEP] he bought a gallon [MASK] milk [SEP]`

Метка: `IsNext`

Отрицательные примеры представляют собой предложения, которые выбираются случайно.

Вход: `[CLS] the man [MASK] to the store [SEP] penguin [MASK] are flight ##less birds [SEP]`

Метка: `NotNext`

Обучение происходит по двум задачам параллельно. Значение функции потерь считается отдельно для маскированного языкового моделирования по токену `[MASK]` и для предсказания следующего слова по токену `[CLS]`.

# Библиотека Transformers

Библиотека [Transformers 🛠️[doc]](https://huggingface.co/docs/transformers/index) создана сообществом HuggingFace — это платформа и сообщество для разработки и обмена моделями машинного обучения в области обработки естественного языка (и не только). Здесь можно найти готовые модели, узнать об их параметрах и применении, а также делиться своими разработками и идеями с другими специалистами. Библиотека [Transformers 🛠️[doc]](https://huggingface.co/docs/transformers/index) позволяет работать с открытыми трансформерными моделями.

In [ ]:
!pip install -q transformers

В библиотеке реализованы классы для различных архитектур — в том числе для модели [BERT 🛠️[doc]](https://huggingface.co/docs/transformers/model_doc/bert).

Нам понадобятся классы `BertTokenizer` [🛠️[doc]](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer) и `BertForMaskedLM` [🛠️[doc]](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForMaskedLM). Для загрузки конкретных моделей используется метод `.from_pretrained()`.

BERT — это общее название архитектуры. С её использованием были обучены модели на различных языках и датасетах. Все модели, которые доступны на HuggingFace, можно посмотреть в разделе [Models 🛠️[doc]](https://huggingface.co/models). Чтобы загрузить модель, нужно указать её идентификатор.

### BertTokenizer

Загрузим токенизатор для модели [BERT base cased 🛠️[doc]](https://huggingface.co/bert-base-cased) для английского языка.

In [ ]:
from transformers import BertTokenizer
from IPython.display import clear_output

en_tz = BertTokenizer.from_pretrained("google-bert/bert-base-cased")
clear_output()

Токенизируем английское предложение с помощью метода `.tokenize()`.

In [ ]:
sent = "He remains characteristically confident and optimistic."
tokenized_sent = en_tz.tokenize(sent)
tokenized_sent

Если какое-то слово не представлено в словаре целиком, при токенизации оно делится на подслова.

Посмотрим, какие индексы в словаре соответствуют словам, с помощью метода `convert_tokens_to_ids()`.

In [ ]:
en_tz.convert_tokens_to_ids(tokenized_sent)

Загрузим токенизатор для модели на основе архитектуры BERT для другого языка (не английского) и попробуем подобрать предложение, где при токенизации одно или более слов делятся на подслова.

Пример для русского языка:

In [ ]:
ru_tz = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
clear_output()

In [ ]:
ru_tz.tokenize("Сельскохозяйственно-машиностроительный"), ru_tz.tokenize(
    "Частнопредпринимательский"
)

Пример для немецкого языка:

In [ ]:
de_tz = BertTokenizer.from_pretrained("google-bert/bert-base-german-cased")
clear_output()

In [ ]:
de_tz.tokenize("die Aufmerksamkeitsdefizitstörung"), de_tz.tokenize("das Ampelmännchen")

### BertForMaskedLM

Загрузим саму модель [BERT base cased 🛠️[doc]](https://huggingface.co/bert-base-cased) для английского языка.

In [ ]:
from transformers import BertForMaskedLM

en_model = BertForMaskedLM.from_pretrained("google-bert/bert-base-cased")
clear_output()

Поскольку модель обучалась на задаче маскированного языкового моделирования, она способна предсказывать наиболее вероятные слова на месте спецтокена `[MASK]`.

Напишем функцию `predict_mask`, которая находит распределение вероятностей для маски.
- Добавим спецтокены `[CLS]` и `[SEP]`
- Токенизируем текст (`text`)
- Определим индекс маскированного слова
- Переведем токенизированные слова (`tokenized_text`) в индексы
- Запишем индексы в тензор
- Применим модель к токенизированному предложению
- Запишем выходы модели для каждого слова
- Применим Softmax (`torch.softmax()`) к результатам для маскированного слова, которое найдем среди всех выходов модели (`predictions`) по индексу (`masked_index`)
- Запишем k самых больших значений весов и их индексы, которые соответствуют словам в словаре
- Пройдем в цикле по списку индексов:
  - Переведем каждый индекс в соответствующий токен
  - Запишем его вероятность

In [ ]:
import torch


def predict_mask(tokenizer, model, text, top_k=5):

    text = f"[CLS] {text} [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs["logits"].squeeze()  # token_len x vocabulary_size

    probs = torch.softmax(predictions[masked_index, :], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print(
            "[MASK]: '%s'" % predicted_token,
            " | weights:",
            round(token_weight.item(), 3),
        )

In [ ]:
predict_mask(en_tz, en_model, "My [MASK] is so cute.", top_k=5)

Загрузим модель на основе архитектуры BERT для другого языка (не английского) и применим функцию к предложению, где одно слово маскировано.

Пример для русского языка:

In [ ]:
ru_model = BertForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased")
clear_output()

In [ ]:
predict_mask(ru_tz, ru_model, "Моя [MASK] очень милая.", top_k=5)

Пример для немецкого языка:

In [ ]:
de_model = BertForMaskedLM.from_pretrained("google-bert/bert-base-german-cased")
clear_output()

In [ ]:
predict_mask(de_tz, de_model, "Meine [MASK] ist sehr nett.", top_k=5)

# Другие модели-энкодеры и их сравнение

После появления модели BERT стали появляться другие модели на основе энкодера трансформера.

**Модель RoBERTa**

[[paper] 🎓 Liu Y. et al. (2019). RoBERTa: A Robustly Optimized BERT Pretraining Approach](https://arxiv.org/abs/1907.11692)

Является улучшенной версией модели BERT за счет более тщательного подбора гиперпараметров.
- Больший объем обучающих данных (в 10 раз больше — с 16 Гб до 160 Гб).
- Увеличение размера батча (с 256 до 8 000) и расширение словаря (с 30 000 до 50 000 слов).
- Динамическое маскирование: для каждого предложения используется 10 разных способов маскирования, через каждые 4 прохода по последовательности меняется позиция маскируемого токена, что помогает модели лучше понимать контекст.
- Модель обучается только на задаче маскированного языкового моделирования, предсказание следующего предложения исключается.

**Модель ALBERT**

[[paper] 🎓 Lan Zh. et al. (2020). ALBERT: A Lite BERT for Self-supervised Learning of Language Representations](https://arxiv.org/abs/1909.11942)

Сокращает количество параметров по сравнению с моделью BERT без снижения качества.
- Факторизованная параметризация эмбеддинга

 Для модели BERT E = H (E — размер эмбеддингов, H — размер скрытого слоя). Слой эмбеддингов имеет размер V × E (V — размер словаря). При увеличении размера скрытого слоя  увеличивается размер слоя эмбеддингов. Для BERT-base E = H = 768, для BERT-large E = H = 1024

 Чтобы размер скрытого слоя и размерность эмбеддинга были разными, после слоя эмбеддингов (V × E) добавляется полносвязный слой (E × H). Позволяет увеличить размер скрытого слоя, не меняя фактического размера эмбеддинга. Для модели ALBERT E = 128, H = 4096.

- Обмен параметрами между слоями

 Архитектуры сетей на основе Трансформера полагаются на независимость слоев. Однако было замечено, что нейросеть выучивается выполнять схожие операции на разных слоях. Эта возможная избыточность устраняется в ALBERT с помощью обмена параметрами между слоями множественного внимания и полносвязными слоями.

- Определение порядка предложений

 Вместо предсказания следующего предложения используется новая задача: определение порядка предложений (Sentence Order Prediction, SOP).  Положительные примеры — два последовательных предложения, отрицательные примеры — те же, но с их измененным порядком.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/BERT_fine_tuning.png" width="600"></center>

<center><em>Источник: <a href="https://arxiv.org/abs/1810.04805">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding</a></em></center>

Как же сравнивать качество работы моделей?

Модели на основе энкодера используются для того, чтобы получать контекстуализированные векторные представления слов и предложений. Далее предобученные модели применяются для решения различных задач:

- классификация пары предложений
  - [[doc] 🛠️ Multi-Genre Natural Language Inference (MultiNLI)](https://cims.nyu.edu/~sbowman/multinli/) — определение логической связи между текстами (для двух утверждений A и B выяснить, следует ли B из A)
  - [[doc] 🛠️ Microsoft Research Paraphrase Corpus (MRPC)](https://www.microsoft.com/en-us/download/details.aspx?id=52398) — определить, являются ли предложения парафразами (выражают одинаковый смысл)
- классификация одного предложения
  - [[doc] 🛠️ Corpus of Linguistic Acceptability (CoLA)](https://nyu-mll.github.io/CoLA/) — бинарная классификация предложений по приемлемости (приемлемые, неприемлемые)
  - [[doc] 🛠️ Stanford Sentiment Treebank (SST)](https://nlp.stanford.edu/sentiment/index.html) — бинарная классификация по тональности (позитивные, негативные)
- поиск ответа на вопрос
  - [[doc] 🛠️ Stanford Question Answering Dataset (SQuAD)](https://rajpurkar.github.io/SQuAD-explorer/) — выделить в тексте подпоследовательность, которая является ответом на заданный вопрос
- теггирование последовательности
  - [[doc] 🛠️ CoNLL-2003](https://aclanthology.org/W03-0419.pdf) — распознавание именованных сущностей (имен людей, названий организаций, топонимов и т.п.)

Наборы для тестирования моделей представлены в бенчмарках [GLUE 🛠️[doc]](https://gluebenchmark.com/) и [SuperGLUE 🛠️[doc]](https://super.gluebenchmark.com/) для английского языка. Для оценки русскоязычных моделей существует бенчмарк [Russian SuperGLUE 🛠️[doc]](https://russiansuperglue.com/).

# Тонкая настройка BERT

Библиотека Transformers от Hugging Face позволяет скачивать предобученные модели и использовать их как начальный блок для подсчета контекстных векторов слов. Поверх этого блока добавляются другие слои, их архитектура зависит от задачи. Веса трансформерных моделей предобучены, но мы проводим тонкую настройку (fine-tuning) или дообучение на целевой задаче.

Мы рассмотрим, как можно осуществлять дообучение модели BERT для классификации предложений. Для этого мы изучим:

- Как подготовить датасет
- Как использовать высокоуровневое API для дообучения модели
- Как использовать собственный цикл обучения (training loop)

Установите библиотеки [Transformers 🛠️[doc]](https://huggingface.co/docs/transformers/index), [Datasets 🛠️[doc]](https://huggingface.co/docs/datasets/index) и [Evaluate 🛠️[doc]](https://huggingface.co/docs/evaluate/index).

In [ ]:
from IPython.display import clear_output

!pip install -q datasets==3.6.0 evaluate transformers
clear_output()

## Предобработка данных

В данном разделе мы будем использовать датасет [Microsoft Research Paraphrase Corpus (MRPC) 🛠️[doc]](https://www.microsoft.com/en-us/download/details.aspx?id=52398). Он состоит из 5801 пары предложений с соответствующим им лейблом: является пара преложений парафразами или нет (т.е. идет ли речь в обоих предложениях об одном и том же). Мы выбрали именно этот датасет, потому что он небольшой: с ним легко экспериментировать в процессе обучения.

### Загрузка датасета

[Hugging Face 🛠️[doc]](https://huggingface.co/) содержит не только модели, там также расположено множество [датасетов 🛠️[doc]](https://huggingface.co/datasets) для различных языков и задач.

Но сейчас вернемся к датасету MRPC! Это один из 10 датасетов бенчмарка [GLUE 🛠️[doc]](https://gluebenchmark.com/) для оценки моделей машинного обучения в задачах классификации текста.

Мы можем загрузить датасет следующим образом:

In [ ]:
from datasets import load_dataset

raw_mrpc = load_dataset("glue", "mrpc")
clear_output()

raw_mrpc

Как можно заметить, мы получили объект типа `DatasetDict`, который включает обучающую выборку, валидационную выборку и тестовую выборку. Каждая из них содержит несколько колонок (`sentence1`, `sentence2`, `label` и `idx`) и переменную с числом элементов (`num_rows`): 3668 пар предложений в обучающей части, 408 в валидационной и 1725 в тестовой .

Мы можем получить доступ к предложениями в объекте `raw_datasets` путем индексирования, как в словаре:

In [ ]:
raw_train_mrpc = raw_mrpc["train"]
print("First element in train dataset:\n")
raw_train_mrpc[0]

Можно увидеть, что лейблы уже являются целыми числами (integer), их обрабатывать не нужно. Чтобы сопоставить индекс класса с его названием, можно вывести значение переменной `features` у `raw_train_dataset`:

In [ ]:
print("Feature types:\n")
raw_train_mrpc.features

Переменная `label` типа `ClassLabel` соответствует именам в `names`. `0` соответствует `not_equivalent`, `1` соответствует `equivalent`.

### Предобработка датасета

Чтобы предобработать датасет, нам необходимо конвертировать текст в числа, которые может обработать модель. Как мы видели ранее, это делается с помощью токенизатора. Мы можем подать на вход токенизатору одно предложение или список, т.е. можно токенизировать предложения попарно таким образом:

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_mrpc["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_mrpc["train"]["sentence2"])

Однако мы не можем просто передать две последовательности в модель и получить прогноз того, являются ли эти два предложения парафразами или нет. Нам нужно обрабатывать две последовательности как пару и применять соответствующую предварительную обработку. К счастью, токенизатор также может взять пару последовательностей и подготовить их так, как ожидает наша модель BERT:

In [ ]:
sentence1 = "This is the first sentence."
sentence2 = "This is the second one."
inputs = tokenizer(sentence1, sentence2)
print(f"First sentence: {sentence1}")
print(f"Second sentence: {sentence2}")
print("Result of tokeniization:")
inputs

`input_ids` содержит индексы, соответствующие токенам по словарю.

Маски внимания (`attention_mask`) — это тензоры той же формы, что и тензор входных идентификаторов, заполненные `0` и `1`: `1` означает, что соответствующие токены должны “привлекать внимание”, а `0` означает, что соответствующие токены не должны “привлекать внимание” (т.е. должны игнорироваться слоями внимания модели).

`token_type_ids` указывает модели, какая часть входных данных является первым предложением, а какая вторым.

Если мы декодируем ID из `input_ids` обратно в слова, мы получим:

In [ ]:
print("Converting indices to tokens:")
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

Токенизатор уже содержит индексы для спецсимволов:
- `[SEP]` — метка конца предложения
- `[CLS]` — токен для классификации предложения
- `[PAD]` — токен для выравнивания длин последовательностей

Видно, что модель ожидает входные данные в следующем формате: `[CLS] sentence1 [SEP] sentence2 [SEP]` в случае двух предложений. Посмотрим соответствие элементов и `token_type_ids`.

In [ ]:
for token, id in zip(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"]), inputs["token_type_ids"]
):
    print(token, id)

Как вы можете заметить, части входных данных, соответствующих `[CLS] sentence1 [SEP]`, имеют тип токена `0`, в то время как остальные части, соответствующие второму предложению `sentence2 [SEP]`, имеют тип токена `1`.

Обратите внимание, что если вы выберете другой идентификатор модели, `token_type_ids` необязательно будут присутствовать в ваших токенизированных входных данных. Они возвращаются только тогда, когда модель будет знать, что с ними делать, потому что она видела их во время предобучения.

В данном случае BERT был обучен с информацией об идентификаторах типов токенов, и помимо задачи маскированного языкового моделирования, он может решать еще одну задачу: предсказание следующего предложения (next sentence prediction). Суть этой задачи — смоделировать связь между предложениями.

В этой задаче модели на вход подаются пары предложений (со случайно замаскированными токенами), от модели требуется предсказать, является ли следующее предложение продолжением текущего. Чтобы задача не была слишком тривиальной, половину времени модель обучается на соседних предложениях из одного документа, другую половину — на парах предложений, взятых из разных источников.

В общем случае вам не нужно беспокоиться о наличии `token_type_ids` в ваших токенизированных данных: пока вы используете одинаковый идентификатор и для токенизатора, и для модели – токенизатор будет знать, как нужно обработать данные.

Теперь мы знаем, что токенизатор может подготовить сразу пару предложений, а значит мы можем использовать его для целого датасета: можно подать на вход токенизатору список первых предложений и список вторых предложений. Это также сработает и для механизмов выравнивания (`padding`) и усечения до максимальной длины (`truncation`).

Итак, один из способов предобработать обучающий датасет:

In [ ]:
tokenized_mrpc = tokenizer(
    raw_mrpc["train"]["sentence1"],
    raw_mrpc["train"]["sentence2"],
    padding=True,
    truncation=True,
)

У данного способа есть недостаток: токенизатор возвращает объект с ключами, `input_ids`, `attention_mask`, и `token_type_ids` и значениями в формате списка списков. Это будет работать, только если у нас достаточно оперативной памяти (RAM) для хранения целого датасета во время токенизации.

In [ ]:
for k, v in tokenized_mrpc.items():
    print(f"Key: {k}, value type: {type(v)}")

Чтобы хранить данные в формате датасета, мы будем использовать метод `Dataset.map()`. Метод `map()` применяет некоторую функцию к каждому элементу датасета.

Давайте определим функцию, которая токенизирует наши входные данные:

In [ ]:
def tokenize_function_mrpc(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

Эта функция принимает на вход словарь (похожий на элементы нашего словаря) и возвращает новый словарь с ключами `input_ids`, `attention_mask` и `token_type_ids`. Заметьте, это также работает, если словарь `example` содержит несколько элементов (каждый ключ в виде списка предложений), поскольку `tokenizer` работает и со списками пар предложений, как мы и видели ранее. Это позволит нам использовать аргумент `batched=True` в вызове `map()`, который ускорит процесс токенизации.

Обратите внимание, что мы оставили аргумент `padding` пустым, потому что дополнение данных до максимальной длины неэффективно: гораздо быстрее делать это во время формирования батча, в таком случае мы будем дополнять до максимальной длины только элементы батча, а не целого датасета. Это поможет сэкономить время в случае длинных последовательностей.

Ниже пример того, как мы применяем функцию токенизации к целому датасету. Поскольку мы указываем `batched=True` при вызове `map`, функция будет применена сразу к нескольким элементам датасета одновременно, а не к каждому по отдельности. Это позволяет сделать токенизацию более быстрой.

In [ ]:
tokenized_mrpc = raw_mrpc.map(tokenize_function_mrpc, batched=True)
clear_output()

tokenized_mrpc

### Dynamic padding

Функция, отвечающая за объединение элементов внутри батча, называется `collate_function`. Это аргумент, который вы можете передать при создании `DataLoader`. По умолчанию это функция, которая просто преобразовывает объекты в тензоры PyTorch и объединяет их. В нашем случае это невозможно, поскольку входные данные, которые у нас есть, не будут иметь одинакового размера. Мы намеренно не стали делать `padding`, чтобы применять его только по мере необходимости в каждом батче и избегать слишком длинных входных данных с большим количеством отступов.

Функция `collate_function` будет осуществлять корректное дополнение элементов выборки, которые мы хотим объединить в батч. Библиотека [Transformers 🛠️[doc]](https://huggingface.co/docs/transformers/index) предоставляет эту функцию через класс `DataCollatorWithPadding`. При создании экземпляра требуется указать токенизатор: чтобы знать, какой токен использовать для дополнения и слева или справа нужно дополнять данные.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Чтобы протестировать, давайте возьмем несколько элементов обучающей выборки, которые мы хотим объединить в батч. Мы удалим колонки `idx`, `sentence1` и `sentence2`, т.к. они содержат строки (а мы не можем превратить строки в тензоры) и посмотрим на длину каждой записи в батче:

In [ ]:
samples = tokenized_mrpc["train"][:8]
samples = {
    k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]
}
[len(x) for x in samples["input_ids"]]

Неудивительно: мы получили объекты разной длины от 32 до 67. Динамическое дополнение подразумевает, что все объекты будут дополнены до максимальной длины в батче, то есть до 67.

Без динамического дополнения все предложения должны быть дополнены до максимальной длины во всем наборе данных, или до максимальной длины, которую может принять модель.

Давайте проверим, что `data_collator` динамически правильно дополняет батч:

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

Выглядит неплохо! Теперь мы пришли от обычного текста к батчу, с которым может работать наша модель.

Можем приступить к тонкой настройке!

## Тонкая настройка модели с использованием Trainer API

Библиотека [Transformers 🛠️[doc]](https://huggingface.co/docs/transformers/index) предоставляет класс `Trainer`, который помогает произвести тонкую настройку любой предобученной модели на вашем датасете. После предобработки данных, сделанной в прошлом разделе, останется сделать несколько шагов для определения `Trainer`.

### Обучение

Первый шаг перед определением `Trainer` — задание класса `TrainingArguments`, который будет содержать все гиперпараметры для `Trainer` (для процессов обучения и валидации). Единственный аргумент, который обязательно нужно задать, — это каталог, в котором будет сохранена обученная модель, а также контрольные точки. Для всего остального можно оставить значения по умолчанию.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="./results", report_to="none")

Второй шаг – задание модели. Мы будем использовать класс `AutoModelForSequenceClassification` с двумя лейблами:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

После создания экземпляра предобученной модели будет выведено предупреждение. Это происходит потому, что BERT не был предобучен для задачи классификации пар предложений, его последний слой не будет использован, вместо него будет добавлен слой, позволяющий работать с такой задачей. Предупреждения сообщают, что некоторые веса не будут использованы, и для новых будут инициализированы случайные веса. В заключении предлагается обучить модель, что мы и сделаем прямо сейчас.

После того как мы загрузили модель, мы можем определить `Trainer` и передать туда нужные объекты: `model`, `training_args`, обучающую и валидационную выборки, `data_collator` и `processing_class`.

In [ ]:
from transformers import Trainer

trainer_mrpc = Trainer(
    model,
    training_args,
    train_dataset=tokenized_mrpc["train"],
    eval_dataset=tokenized_mrpc["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

Для тонкой настройки модели на нашем датасете нужно вызвать метод `train()` у `Trainer`:

In [ ]:
trainer_mrpc.train()

Это запустит процесс дообучения, который должен занять несколько минут на GPU и будет выводить значение функции потерь каждые 500 итераций. Однако эти значения не скажут нам, насколько хорошо или плохо модель работает. И вот почему:

1. Мы не сообщили `Trainer`, что необходимо проводить валидацию: для этого нужно присвоить аргументу `evaluation_strategy` значение `"steps"` (валидировать каждые `eval_steps`) или `"epoch"` (валидировать по окончании каждой эпохи).
2. Мы не указали `Trainer` аргумент `compute_metrics()` – функцию для вычисления метрики на валидационной части. В таком случае в процессе валидации будет только выводиться значение функции потерь, что не очень информативно.

### Валидация

Посмотрим, как мы можем создать и использовать в процессе обучения полезную функцию `compute_metrics()`. Функция должна принимать на вход объект `EvalPrediction` (именованный кортеж с полями `predictions` и `label_ids`) и возвращать словарь, где ключи — названия метрик, а значения — оценки этих метрик.

Чтобы получить предсказания, мы можем использовать функцию `Trainer.predict()`:

In [ ]:
predictions_mrpc = trainer_mrpc.predict(tokenized_mrpc["validation"])
print(f"First 5 elements in predicted labels:\n{predictions_mrpc.predictions[:5]}")
print(f"Shape of predicted labels: {predictions_mrpc.predictions.shape}")
print(f"First 5 elements in true labels: {predictions_mrpc.label_ids[:5]}")
print(f"Shape of true labels: {predictions_mrpc.label_ids.shape}")
print(f"Metrics: {predictions_mrpc.metrics}")

Результат функции `predict()` — именованный кортеж с полями `predictions`, `label_ids` и `metrics`. Поле `metrics` содержит значение функции потерь на нашем датасете и значения метрик. После реализации функции `compute_metrics()` и передачи ее в `Trainer` поле `metrics` также будет содержать результат функции `compute_metrics()`.

Как можно заметить, `predictions` — массив размером 408 × 2 (408 — число элементов в датасете, который мы использовали). Это логиты для каждого элемента нашего датасета, переданного `в predict()`. Чтобы превратить их в предсказания и сравнить с нашими лейблами, нам необходимо узнать индекс максимального элемента второй оси:

In [ ]:
import numpy as np

preds_mrpc = np.argmax(predictions_mrpc.predictions, axis=-1)
print(f"First 5 elements in predicted labels after argmax:\n{preds_mrpc[:5]}")
print(f"Shape: {preds_mrpc.shape}")

Теперь мы можем сравнить эти предсказания с истинными метками. Для создания функции `compute_metric()` мы воспользуемся метриками из библиотеки [Evaluate 🛠️[doc]](https://huggingface.co/docs/evaluate/index). Мы можем загрузить подходящие для датасета MRPC метрики так же просто, как мы загрузили датасет, но на этот раз с помощью функции `evaluate.load()`. Возвращаемый объект имеет метод `compute()`, который мы можем использовать для вычисления метрики:

In [ ]:
import evaluate

metric_mrpc = evaluate.load("glue", "mrpc")
metric_mrpc.compute(predictions=preds_mrpc, references=predictions_mrpc.label_ids)

Эти метрики используются для валидации результатов на MRPC датасете на бенчмарке GLUE. В таблице из статьи о BERT указано значение F1 оценки в 88.9 для базовой модели. Это была оценка для варианта модели `uncased`, а мы использовали `cased`, этим и объясняется более хороший результат.

Собирая вместе все фрагменты выше, мы получим нашу функцию `compute_metrics()`:

In [ ]:
def compute_metrics_mrpc(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Чтобы увидеть эту функцию в действии после каждой эпохи, ниже мы определим еще один `Trainer` с функцией `compute_metrics()`:

In [ ]:
training_args = TrainingArguments(
    output_dir="./results", eval_strategy="epoch", report_to="none"
)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer_mrpc = Trainer(
    model,
    training_args,
    train_dataset=tokenized_mrpc["train"],
    eval_dataset=tokenized_mrpc["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics_mrpc,
)

Обратите внимание, что мы создали новый объект `TrainingArguments` с новой моделью — иначе мы бы продолжили обучать модель, которая уже является обученной. Чтобы запустить обучение заново, надо выполнить:

In [ ]:
trainer_mrpc.train()

На этот раз будут выводиться значение функции потерь и метрики на валидации по окончании каждой эпохи обучения.

## Полное обучение модели

Теперь посмотрим, как достичь результатов из предыдущей части без использования класса `Trainer`.

### Подготовка к обучению

Перед реализацией цикла обучения необходимо задать несколько объектов. Первый: загрузчики данных (далее — `dataloaders`), которые мы будем использовать для итерирования по батчам данных. Перед этим нам необходимо применить несколько операций предобработки к нашему `tokenized_datasets`. В прошлый раз за нас это автоматически делал `Trainer`. Необходимо сделать следующее:
- Удалить колонки, соответствующие значениям, которые модель не принимает на вход (например, `sentence1` и `sentence2`).
- Переименовать колонку `label` в `labels` (потому что модель ожидает аргумент, названный `labels`).
- Задать тип данных в датасете `pytorch tensors` вместо списков.

Наш `tokenized_datasets` предоставляет возможность использовать встроенные методы для каждого из приведенных выше шагов. Мы можем проверить, что в результате у нас присутствуют только те поля, которые ожидает наша модель:

In [ ]:
tokenized_mrpc = tokenized_mrpc.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_mrpc = tokenized_mrpc.rename_column("label", "labels")
tokenized_mrpc.set_format("torch")
tokenized_mrpc["train"].column_names

Теперь, когда датасет готов, мы можем задать `dataloader`:

In [ ]:
from torch.utils.data import DataLoader

train_dataloader_mrpc = DataLoader(
    tokenized_mrpc["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader_mrpc = DataLoader(
    tokenized_mrpc["validation"], batch_size=8, collate_fn=data_collator
)

Для того чтобы убедиться в отсутствии ошибок в сделанной нами предобработке, мы можем проверить один батч данных:

In [ ]:
for batch in train_dataloader_mrpc:
    break
{k: v.shape for k, v in batch.items()}

Обратите внимание, что фактические размеры, вероятно, будут немного отличаться в вашем случае, так как мы установили `shuffle=True` для обучающего загрузчика данных, также мы дополняем (padding) до максимальной длины внутри батча.

Теперь мы полностью завершили этап предобработки, перейдем к модели. Мы инициализируем ее точно так же, как в предыдущем примере:

In [ ]:
from transformers import AutoModelForSequenceClassification

model_mrpc = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2
)

Чтобы убедиться, что обучение пойдет гладко, подадим на вход модели один батч:

In [ ]:
outputs_mrpc = model_mrpc(**batch)
print(outputs_mrpc.loss, outputs_mrpc.logits.shape)

Все модели трансформеров возвращают значение функции потерь, если в данных были `labels`, а также логиты (в результате получается тензор 8 × 2).

Мы почти готовы к написанию обучающего цикла! Мы пропустили только две вещи: оптимизатор и планировщик скорости обучения (learning rate scheduler). Ввиду того, что мы пытаемся повторить вручную то, что делал за нас `Trainer`, мы будем использовать такие же значения по умолчанию. Оптимизатор, используемый в Trainer, — `AdamW`, который является почти полной копией Adam, за исключением добавления регуляризации (weight decay). Она представляет собой штраф за высокие значения весов нейронной сети.

In [ ]:
from torch.optim import AdamW

optimizer_mrpc = AdamW(model_mrpc.parameters(), lr=5e-5)

Наконец, необходимо определить планировщик скорости обучения, который уменьшает скорость обучения для улучшения сходимости и качества модели. При типе `"linear"` скорость обучения линейно уменьшается от начального значения до 0 за заданное количество шагов. Оно определяется как произведение числа эпох и числа батчей (длины нашего загрузчика данных).

Число эпох по умолчанию в `Trainer` равно 3, так же мы зададим его и сейчас:

In [ ]:
from transformers import get_scheduler

num_epochs_mrpc = 3
num_training_steps_mrpc = num_epochs_mrpc * len(train_dataloader_mrpc)
lr_scheduler_mrpc = get_scheduler(
    "linear",
    optimizer=optimizer_mrpc,
    num_warmup_steps=0,
    num_training_steps=num_training_steps_mrpc,
)
print(num_training_steps_mrpc)

### Обучающий цикл

Последний момент: мы хотим использовать GPU в случае, если у нас будет такая возможность (на CPU процесс может занять несколько часов вместо пары минут). Чтобы добиться этого, мы определим переменную `device` и «прикрепим» к видеокарте нашу модель и данные.

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_mrpc.to(device)
device

Теперь мы готовы к обучению модели! Чтобы иметь представление о том, сколько времени это может занять, мы добавим индикатор прогресса, который будет иллюстрировать, сколько шагов обучения уже выполнено. Это можно сделать с использованием бибилиотеки [tqdm 🛠️[doc]](https://tqdm.github.io/):

In [ ]:
from tqdm.auto import tqdm

progress_bar_mrpc = tqdm(range(num_training_steps_mrpc))

model_mrpc.train()
for epoch in range(num_epochs_mrpc):
    for batch in train_dataloader_mrpc:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_mrpc(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer_mrpc.step()
        lr_scheduler_mrpc.step()
        optimizer_mrpc.zero_grad()
        progress_bar_mrpc.update(1)

Вы можете заметить, что процесс обучения выглядит очень похожим на то, как он выглядел в наших первых примерах. Мы не указывали модели, чтобы она нам что-то возвращала в процессе обучения. Для этого мы добавим цикл валидации.

### Валидационный цикл

Ранее мы использовали метрику, которую нам предоставляла библиотека [Evaluate 🛠️[doc]](https://huggingface.co/docs/evaluate/index). Мы уже знаем, что есть метод `metric.compute()`, однако метрики могут накапливать значения в процессе итерирования по батчу, для этого есть метод `add_batch()`. После того как мы пройдемся по всем батчам, мы сможем вычислить финальный результат с помощью `metric.compute()`. Вот пример того, как это можно сделать в цикле валидации:

In [ ]:
import evaluate

metric_mrpc = evaluate.load("glue", "mrpc")
model_mrpc.eval()
for batch in eval_dataloader_mrpc:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_mrpc(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric_mrpc.add_batch(predictions=predictions, references=batch["labels"])

metric_mrpc.compute()

## Тонкая настройка для других задач

Помимо классификации предложений, языковая модель на основе энкодера Трансформера может быть дообучена для таких задач, как классификация каждого токена и поиск ответа на вопрос.

Примером первой задачи является разметка текстов, в частности, выделение именованных сущностей (**Named Entity Recognition**, ***NER***):

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L06/out/ner.png" width="800"></center>

В случае второй задачи, по-английски называемой ***Question Answering (QA)***, ответ **извлекается** (extract) из контекста. Контекст не может быть пустым.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L06/qa_example.png" width="500"></center>

<center><em>Source: <a href="https://www.deleeuw.me.uk/posts/Using-PrimeQA-For-NLP-Question-Answering/">Using PrimeQA For NLP Question Answering</a></em></center>

Примеры тонкой настройки для данных задач можно найти в тьюториалах Hugging Face:
- [[doc] 🛠️ Token classification](https://huggingface.co/learn/nlp-course/en/chapter7/2?fw=pt)
- [[doc] 🛠️ Question answering](https://huggingface.co/learn/nlp-course/en/chapter7/7?fw=pt)